In [1]:
using LinearAlgebra

# Implementation of simplex algorithm

The simplex algorithm is a optimization algorithm for linear problems. In our implementation we must create a method to the user input the linear problem and a method for the optimization procedure.

The problems solved are in the following format:

$$\max x^Tc$$

$$\text{subject to } $$

$$A_{ub}x \leq b_{ub}$$

$$A_{eq}x = b_{eq}$$

$$x \geq 0$$

The matrix $c$, $A_{ub}$, $b_{ub}$, $A_{eq}$ and $b_{eq}$ are the input from the user. The simplex method can be described in the following steps:

- Add slacks variables for upper bound constraints.

- For every constraint where $b$ is lower than $0$, multiply the constraint by $-1$ to make it positive.

- Add artificial variables for every row where there isn't a isolated variable.

- If there are artificial variables, perform phase I optimization.

- With there aren't artificial variables or after phase II, perform phase II optimization.
    
The optimization steps are the following until it is possible to increase the solution:
    
- Find variable if biggest coefficient on objective function.

- Find row with lowest upper limit for the variable.

- Add new variable to basis and remove the older one.

To formulate operations we will use a tableau, a tableau is a matrix that contain the matrix of restrictions $A$, the matrix of the right of the side of restrictions $b$ and the objective function $c$. The tableau will also keep infromation about decision, slack and artificial variables, the constraints and the basis. Obs.: the basis is the list of variables that are not $0$. We formulate the struct:
    

In [2]:
mutable struct Tableau
    T::Array{Float64, 2}
    basis::Dict{Int64,Int64}
    decision::Array{Int64, 1}
    slack::Array{Int64, 1}
    artificial::Array{Int64, 1}
    constraints::Array{Int64, 1}
end

The first step if to add slack variables for each upper bound constraint, we add a empty column with the value $1$ on the row of the respective constraint. We also keep the index for the slack variables created.

In [3]:
function add_slack(T::Array{Float64, 2},
                    n_ub_const::Int64,
                    n_variables::Int64)
    """
    Function that add slack variables to tableau
    """
    aux = zeros(size(T, 1), n_ub_const)
    aux[1:n_ub_const, 1:n_ub_const] = Matrix(I(n_ub_const))    
    T = [T[:, 1:(end- 1)] aux T[:, end]]
    slack = collect((n_variables + 1):(size(T, 2) - 1))
    return T, slack
end

add_slack (generic function with 1 method)

To keep track of the problem, we start creating the basis with the slack variables that have a positive right side.

In [4]:
function slack_basis(T::Array{Float64, 2},
                    n_ub_const::Int64,
                    n_variables::Int64)
    """
    Function that add every slack variable where the right side if positive to basis 
    """
    basis = Dict{Int64,Int64}()
    for i = 1:n_ub_const
        if T[i, end] >= 0
            push!(basis, i => (i+n_variables))
        end
    end
    return T, basis
end

slack_basis (generic function with 1 method)

The next step is to remove the negative right side from constraints.

In [5]:
function remove_negative(T::Array{Float64, 2},
                    n_ub_const::Int64,
                    n_eq_const::Int64)
    """
    Function that remove negative values from right side of tableau
    """
    for i = 1:(n_ub_const + n_eq_const)
        if T[i, end] < 0
            T[i, :] = -1 *T[i, :]
        end
    end
    return T
end

remove_negative (generic function with 1 method)

Now we need to check if there is need for artificial variables, for every constraint we verify if there is a isolated variable already in basis, if there isn't we check for every column if there is a isolated variable to pivot and add to basis. In the rows that we still couldn't find a isolated variable, we add an artificial one.

In [6]:
function add_artificial(T::Array{Float64, 2},
                    n_ub_const::Int64,
                    n_eq_const::Int64,
                    basis::Dict{Int64, Int64})
    """
    Function that add artificial variables to every row without a isolated variable
    """
    artificial = []
    aux = zeros(1, size(T, 2))
    T = [T; aux]
    for i = 1:(n_ub_const + n_eq_const)
        if !(i in keys(basis))
            row_with_basis = false
            for j=1:(size(T, 2)-1)
                if (sum(abs.(T[1:(n_ub_const + n_eq_const), j])) == abs(T[i, j])) & !(j in values(basis))
                    #normalize row and add to basis
                    T[i, :] = T[i, :]/T[i, j]
                    push!(basis, i => j)
                    row_with_basis = true
                    break
                end
            end
            
            if !row_with_basis
                aux = zeros(size(T, 1))
                aux[i] = 1.0
                T = [T[:, 1:(end- 1)] aux T[:, end]]
                artificial = [artificial; (size(T, 2) - 1)]
                push!(basis, i => last(artificial))
                T[end, :] = T[end, :] + T[i, :]
            end
        end
    end
    
    if size(artificial, 1) == 0
        T = T[1:(end - 1), :]
    else
        T[end, (end - size(artificial, 1)):(end - 1)] .= 0
    end
    return T, basis, artificial
end

add_artificial (generic function with 1 method)

Now we need to make the transition from phase I to phase II. First we check if the problem is feasible, next, we iterate over basis and check if there is artificial variables on basis, if there is, we check if it is possible to remove then by pivoting another variable, if we couldn't we keep them. In the end, we remove every artificial variable that is not from basis from the tableau.

In [7]:
function phaseI_phaseII(tableau::Tableau)
    """
    Function that made transition from phase I to phase II
    It starts cheking if a feasible solutions was reached.
    For each artificial variable, verify if it's a basis, if it isn't, remove column.
    If it is, check if there is another variable on row to pivot.
        
    """
    if tableau.T[end, end] != 0
        error("Problem is infeasible")
    end
    
    artificial_to_remove = copy(tableau.artificial)
    
    for (key, value) in tableau.basis
        if value in tableau.artificial
            non_zero_index = 0
            all_zero = true
            for j in 1:(size(tableau.T, 2) - 1)
                if tableau.T[key, j] != 0
                    all_zero = false
                    non_zero_index = copy(j)
                    break
                end
            end
            
            if !all_zero
                tableau = pivot_tableau(tableau, key, non_zero_index)
            else
                filter!(x -> x != value, artificial_to_remove)
            end
        end
    end
    
    for i in artificial_to_remove
        tableau.T[:, i] .= 0
        filter!(k -> k != i, tableau.artificial)
    end
    
    tableau.T = tableau.T[1:(end - 1), :]
    
    return tableau
end

phaseI_phaseII (generic function with 1 method)

The iterations for the simplex start finding if it's possible to increase the objective, we find the variable with the biggest coefficient and find the upper limit for it. Next we must pivot the tableau to add this new variable to basis.

In [8]:
function simplex_iteration(tableau::Tableau, max_int::Int64)
    """
        Function that run the iterations of optimization for the simplex
    
        tableau - tableau object with problem information
        max_int - number of max iterations
        
    """
    n = size(tableau.T, 1)
    m = size(tableau.T, 2)
    
    for int=1:max_int
        
        #can't increase objective
        if maximum(tableau.T[n, 1:(end - 1)]) <= 0
            return tableau
        end
        
        #variable to enter basis
        s = argmax(tableau.T[n, 1:(end -1)])
        
        #check if is unbounded
        if maximum(tableau.T[tableau.constraints,s]) <= 0
            error("Problem is unbounded.")
        end
        
        #ratio test to find variable to remove from basis
        r = -1
        min_ratio = Inf
        for i in tableau.constraints
            if tableau.T[i, s] > 0
                new_ratio = tableau.T[i, m]/tableau.T[i, s]
                if new_ratio < min_ratio
                    r = i
                    min_ratio = new_ratio
                end
            end
        end
        
        tableau = pivot_tableau(tableau, r, s)
    end
    return tableau
end

simplex_iteration (generic function with 1 method)

To pivot the tableau, we update the basis, normalize the row where the basis was updated and update every other row of the tableau.

In [9]:
function pivot_tableau(tableau::Tableau,
                        r::Int64,
                        s::Int64)
    """
    Function that pivot tableau, removing variable of line r from basis and adding variable s to line r
    """
    
    #update basis variable
    delete!(tableau.basis, r)
    push!(tableau.basis, r => s)   
    
    #normalize row r
    tableau.T[r, :] .= tableau.T[r, :]/tableau.T[r, s]
    
    #subtract from others rows
    for i=1:size(tableau.T, 1)
        if i != r
            tableau.T[i, :] .= tableau.T[i, :] .- (tableau.T[i, s] * tableau.T[r, :])
        end
    end
    
    return tableau
end

pivot_tableau (generic function with 1 method)

The following function obtain the matrix from the user and call the steps.

In [10]:
function linear_program(c::Array{Float64,1},
                        A_ub::Array{Float64,2},
                        b_ub::Array{Float64,1},
                        A_eq::Array{Float64,2},
                        b_eq::Array{Float64,1},
                        max_int::Int64,
                        verbose::Bool)
    
    
    n_variables = size(A_ub, 2)
    n_ub_const = size(A_ub, 1)
    n_eq_const = size(A_eq, 1)
    decision = collect(1:n_variables)
    constraint = collect(1:(n_ub_const + n_eq_const))
    if verbose
        println(string(n_variables) * " decision variables.")
        println(string(n_ub_const + n_eq_const) * " constraints.")
    end

    T = zeros((size(A_ub, 1) + size(A_eq, 1) + 1), (size(A_ub, 2) + 1))
    T[1:(end -1), :] = [A_ub b_ub; A_eq b_eq;]
    T[end, :] = [c; 0]
    
    T, slack = add_slack(T, n_ub_const, n_variables)
    
    if verbose
        println(string(size(slack, 1)) * " slack variables.")
    end
    
    T, basis = slack_basis(T, n_ub_const, n_variables)
    
    if verbose
        println("Start basis:")
        println(basis)
    end
    
    T = remove_negative(T, n_ub_const, n_eq_const)
    
    T, basis, artificial = add_artificial(T, n_ub_const, n_eq_const, basis)
    
    if verbose
        println(string(size(artificial, 1)) * " artificial variables.")
        println("Updated basis:")
        println(basis)
    end
    
        
    tableau = Tableau(T, basis, decision, slack, artificial, constraint) 
    
    if size(artificial, 1) > 0
        
        if verbose
            println("Phase I is needed.")
        end
        tableau = simplex_iteration(tableau, max_int)
        tableau = phaseI_phaseII(tableau)
        if verbose
            println(string(size(tableau.artificial, 1)) * " artificial variables after phase I.")
        end
    end
    
    if verbose
        println("Starting phase II.")
    end
    
    tableau = simplex_iteration(tableau, max_int)
    
    return create_solution(tableau)
end

linear_program (generic function with 1 method)

We also created with two functions that deal with the solution from the tableau.

In [11]:
mutable struct solution
    objective::Float64
    decision::Array{Float64, 1}
end

In [12]:
function create_solution(tableau::Tableau)
    objective = -tableau.T[end, end]
    x = zeros(size(tableau.decision, 1))
    for (key, value) in tableau.basis
        if value in tableau.decision
            x[value] = tableau.T[key, end]
        end
    end
    return solution(objective, x)
end

create_solution (generic function with 1 method)

## Examples

We start with a example where only the phase II is required.

In [13]:
A_ub = [[0.5 2 1];
    [1 2 4]]
b_ub = [24.0, 60.0];
c = [6.0, 14, 13];
A_eq = Array{Float64}(undef, 0, 3);
b_eq = Array{Float64}(undef,0 );
sol = linear_program(c, A_ub, b_ub, A_eq, b_eq, 100, true)
println("Z = " * string(sol.objective))
println("X = " * string(sol.decision))

3 decision variables.
2 constraints.
2 slack variables.
Start basis:
Dict(2 => 5,1 => 4)
0 artificial variables.
Updated basis:
Dict(2 => 5,1 => 4)
Starting phase II.
Z = 294.0
X = [35.99999999999999, 0.0, 6.000000000000002]


Now an example where phase I is required.

In [14]:
A_eq = [[1.0 1 1 1];
    [-2 1 -1 0];
    [0 3 1 1]]
b_eq = [4.0, 1, 9]
c = [-3.0, 0, 1, 0]
A_ub = Array{Float64}(undef,0, 4);
b_ub = Array{Float64}(undef,0 );
sol = linear_program(c, A_ub, b_ub, A_eq, b_eq, 100, true)
println("Z = " * string(sol.objective))
println("X = " * string(sol.decision))

4 decision variables.
3 constraints.
0 slack variables.
Start basis:
Dict{Int64,Int64}()
3 artificial variables.
Updated basis:
Dict(2 => 6,3 => 7,1 => 5)
Phase I is needed.
0 artificial variables after phase I.
Starting phase II.
Z = 1.5
X = [0.0, 2.5, 1.5, 0.0]
